In [1]:
# packages, librerias y dependencias necesarias para el ejercicio
import pandas as pd
import numpy as np
import requests
import os
import datetime
import dateutil
import subprocess
import sys
import json
import tempfile

In [25]:
!pip install unidecode
import unidecode


In [3]:
# datos de entrada disponibles en el directorio "../input/".
direc_input = './'
if os.path.split(os.path.abspath('.'))[-1] == 'src':
    direc_input = '../input'
# datos de salida disponibles en el directorio "../output/".
direc_output = './'
if os.path.split(os.path.abspath('.'))[-1] == 'src':
    direc_output = '../output'

In [2]:
# data oficial
url_dataset = 'https://www.datos.gov.co/api/views/gt2j-8ykr/rows.csv?accessType=DOWNLOAD'
# muestras procesadas diarias
url_muestras_diarias = 'https://e.infogram.com/api/live/flex/638d656c-c77b-4326-97d3-e50cb410c6ab/8188140c-8352-4994-85e3-2100a4dbd9db?'

In [4]:
# crear archivo data_covid.csv
with requests.get(url_dataset) as dataset_oficial:
    with open(os.path.join(direc_output, 'data_covid.csv'), 'wb') as dataset_file:
        dataset_file.write(dataset_oficial.content)

In [6]:
# leer csv - crear dataframe
dfcovid19 = pd.read_csv(os.path.join(direc_output, 'data_covid.csv'))
dfcovid19.shape # tamaño del archivo

(541147, 21)

In [7]:
# ver dataframe
dfcovid19.tail()

,ID de caso,Fecha de notificación,Código DIVIPOLA,Ciudad de ubicación,Departamento o Distrito,atención,Edad,Sexo,Tipo,Estado,...,FIS,Fecha de muerte,Fecha diagnostico,Fecha recuperado,fecha reporte web,Tipo recuperación,Codigo departamento,Codigo pais,Pertenencia etnica,Nombre grupo etnico
541142,541183,2020-08-09T00:00:00.000,5088,Bello,Antioquia,Casa,37,F,En estudio,Leve,...,2020-08-05T00:00:00.000,NaN,2020-08-20T00:00:00.000,NaN,2020-08-23T00:00:00.000,NaN,5,NaN,NaN,NaN
541143,541184,2020-08-09T00:00:00.000,5615,Rionegro,Antioquia,Casa,40,F,En estudio,Leve,...,2020-08-05T00:00:00.000,NaN,2020-08-20T00:00:00.000,NaN,2020-08-23T00:00:00.000,NaN,5,NaN,NaN,NaN
541144,541185,2020-08-09T00:00:00.000,5212,Copacabana,Antioquia,Casa,27,F,En estudio,Leve,...,2020-08-05T00:00:00.000,NaN,2020-08-20T00:00:00.000,NaN,2020-08-23T00:00:00.000,NaN,5,NaN,NaN,NaN
541145,541186,2020-08-09T00:00:00.000,5045,Apartadó,Antioquia,Casa,42,F,En estudio,Leve,...,2020-08-05T00:00:00.000,NaN,2020-08-20T00:00:00.000,NaN,2020-08-23T00:00:00.000,NaN,5,NaN,NaN,NaN
541146,541187,2020-08-09T00:00:00.000,5376,La Ceja,Antioquia,Casa,7,M,En estudio,Leve,...,2020-08-05T00:00:00.000,NaN,2020-08-20T00:00:00.000,NaN,2020-08-23T00:00:00.000,NaN,5,NaN,NaN,NaN


In [8]:
list(dfcovid19.columns.values) # atributos

['ID de caso',
 'Fecha de notificación',
 'Código DIVIPOLA',
 'Ciudad de ubicación',
 'Departamento o Distrito ',
 'atención',
 'Edad',
 'Sexo',
 'Tipo',
 'Estado',
 'País de procedencia',
 'FIS',
 'Fecha de muerte',
 'Fecha diagnostico',
 'Fecha recuperado',
 'fecha reporte web',
 'Tipo recuperación',
 'Codigo departamento',
 'Codigo pais',
 'Pertenencia etnica',
 'Nombre grupo etnico']

In [27]:
# nombre de atributo en mayuscula y sin acento
dfcovid19.columns = [unidecode.unidecode(value).upper() for value in dfcovid19.columns]

# texto en formato de texto de nombre de atributo
#for attr in dfcovid19.columns:
    #if dfcovid19[attr].dtypes == 'object':
        #dfcovid19[attr] = dfcovid19[attr].transform(lambda value: str(value).title())

dfcovid19.head() # ver dataframe

,ID DE CASO,FECHA DE NOTIFICACION,CODIGO DIVIPOLA,CIUDAD DE UBICACION,DEPARTAMENTO O DISTRITO,ATENCION,EDAD,SEXO,TIPO,ESTADO,...,FIS,FECHA DE MUERTE,FECHA DIAGNOSTICO,FECHA RECUPERADO,FECHA REPORTE WEB,TIPO RECUPERACION,CODIGO DEPARTAMENTO,CODIGO PAIS,PERTENENCIA ETNICA,NOMBRE GRUPO ETNICO
0,1,2020-03-02T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,19,F,Importado,Leve,...,2020-02-27T00:00:00.000,Nan,2020-03-06T00:00:00.000,2020-03-13T00:00:00.000,2020-03-06T00:00:00.000,Pcr,11,380.0,Otro,Nan
1,2,2020-03-06T00:00:00.000,76111,Guadalajara De Buga,Valle Del Cauca,Recuperado,34,M,Importado,Leve,...,2020-03-04T00:00:00.000,Nan,2020-03-09T00:00:00.000,2020-03-19T00:00:00.000,2020-03-09T00:00:00.000,Pcr,76,724.0,Otro,Nan
2,3,2020-03-07T00:00:00.000,5001,Medellín,Antioquia,Recuperado,50,F,Importado,Leve,...,2020-02-29T00:00:00.000,Nan,2020-03-09T00:00:00.000,2020-03-15T00:00:00.000,2020-03-09T00:00:00.000,Pcr,5,724.0,Otro,Nan
3,4,2020-03-09T00:00:00.000,5001,Medellín,Antioquia,Recuperado,55,M,Relacionado,Leve,...,2020-03-06T00:00:00.000,Nan,2020-03-11T00:00:00.000,2020-03-26T00:00:00.000,2020-03-11T00:00:00.000,Pcr,5,NaN,Otro,Nan
4,5,2020-03-09T00:00:00.000,5001,Medellín,Antioquia,Recuperado,25,M,Relacionado,Leve,...,2020-03-08T00:00:00.000,Nan,2020-03-11T00:00:00.000,2020-03-23T00:00:00.000,2020-03-11T00:00:00.000,Pcr,5,NaN,Otro,Nan


In [40]:
list(dfcovid19.columns.values) # atributos

['ID DE CASO',
 'FECHA DE NOTIFICACION',
 'CODIGO DIVIPOLA',
 'CIUDAD DE UBICACION',
 'DEPARTAMENTO O DISTRITO ',
 'ATENCION',
 'EDAD',
 'SEXO',
 'TIPO',
 'ESTADO',
 'PAIS DE PROCEDENCIA',
 'FIS',
 'FECHA DE MUERTE',
 'FECHA DIAGNOSTICO',
 'FECHA RECUPERADO',
 'FECHA REPORTE WEB',
 'TIPO RECUPERACION',
 'CODIGO DEPARTAMENTO',
 'CODIGO PAIS',
 'PERTENENCIA ETNICA',
 'NOMBRE GRUPO ETNICO']

In [54]:
#Limpiar Dataframe
# Medellín 05001
# Bogotá 11001
# Barranquilla 08001
# Cali 76001
# Cartagena 13001
dfsincolumnan = dfcovid19.drop(['NOMBRE GRUPO ETNICO', 'PERTENENCIA ETNICA'], axis=1)
dfsincolumnan.head()


,ID DE CASO,FECHA DE NOTIFICACION,CODIGO DIVIPOLA,CIUDAD DE UBICACION,DEPARTAMENTO O DISTRITO,ATENCION,EDAD,SEXO,TIPO,ESTADO,PAIS DE PROCEDENCIA,FIS,FECHA DE MUERTE,FECHA DIAGNOSTICO,FECHA RECUPERADO,FECHA REPORTE WEB,TIPO RECUPERACION,CODIGO DEPARTAMENTO,CODIGO PAIS
0,1,2020-03-02T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,19,F,Importado,Leve,Italia,2020-02-27T00:00:00.000,Nan,2020-03-06T00:00:00.000,2020-03-13T00:00:00.000,2020-03-06T00:00:00.000,Pcr,11,380
1,2,2020-03-06T00:00:00.000,76111,Guadalajara De Buga,Valle Del Cauca,Recuperado,34,M,Importado,Leve,España,2020-03-04T00:00:00.000,Nan,2020-03-09T00:00:00.000,2020-03-19T00:00:00.000,2020-03-09T00:00:00.000,Pcr,76,724
2,3,2020-03-07T00:00:00.000,5001,Medellín,Antioquia,Recuperado,50,F,Importado,Leve,España,2020-02-29T00:00:00.000,Nan,2020-03-09T00:00:00.000,2020-03-15T00:00:00.000,2020-03-09T00:00:00.000,Pcr,5,724
3,4,2020-03-09T00:00:00.000,5001,Medellín,Antioquia,Recuperado,55,M,Relacionado,Leve,Nan,2020-03-06T00:00:00.000,Nan,2020-03-11T00:00:00.000,2020-03-26T00:00:00.000,2020-03-11T00:00:00.000,Pcr,5,-
4,5,2020-03-09T00:00:00.000,5001,Medellín,Antioquia,Recuperado,25,M,Relacionado,Leve,Nan,2020-03-08T00:00:00.000,Nan,2020-03-11T00:00:00.000,2020-03-23T00:00:00.000,2020-03-11T00:00:00.000,Pcr,5,-


In [71]:
dfcovid19.dtypes

ID DE CASO                   int64
FECHA DE NOTIFICACION       object
CODIGO DIVIPOLA              int64
CIUDAD DE UBICACION         object
DEPARTAMENTO O DISTRITO     object
ATENCION                    object
EDAD                         int64
SEXO                        object
TIPO                        object
ESTADO                      object
PAIS DE PROCEDENCIA         object
FIS                         object
FECHA DE MUERTE             object
FECHA DIAGNOSTICO           object
FECHA RECUPERADO            object
FECHA REPORTE WEB           object
TIPO RECUPERACION           object
CODIGO DEPARTAMENTO          int64
CODIGO PAIS                 object
PERTENENCIA ETNICA          object
NOMBRE GRUPO ETNICO         object
dtype: object

In [68]:
# eliminar ciudades menos Bogotá 
dfcovid19_2 = dfcovid19.drop(dfcovid19[(dfcovid19['CODIGO DIVIPOLA'] != 11001)&(dfcovid19['CODIGO DIVIPOLA'] != 76001)].index)
dfcovid19_2.head()

,ID DE CASO,FECHA DE NOTIFICACION,CODIGO DIVIPOLA,CIUDAD DE UBICACION,DEPARTAMENTO O DISTRITO,ATENCION,EDAD,SEXO,TIPO,ESTADO,...,FIS,FECHA DE MUERTE,FECHA DIAGNOSTICO,FECHA RECUPERADO,FECHA REPORTE WEB,TIPO RECUPERACION,CODIGO DEPARTAMENTO,CODIGO PAIS,PERTENENCIA ETNICA,NOMBRE GRUPO ETNICO
0,1,2020-03-02T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,19,F,Importado,Leve,...,2020-02-27T00:00:00.000,Nan,2020-03-06T00:00:00.000,2020-03-13T00:00:00.000,2020-03-06T00:00:00.000,Pcr,11,380,Otro,Nan
7,8,2020-03-09T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,22,F,Importado,Leve,...,2020-03-06T00:00:00.000,Nan,2020-03-11T00:00:00.000,2020-03-21T00:00:00.000,2020-03-11T00:00:00.000,Pcr,11,724,Otro,Nan
8,9,2020-03-08T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,28,F,Importado,Leve,...,2020-03-07T00:00:00.000,Nan,2020-03-11T00:00:00.000,2020-03-23T00:00:00.000,2020-03-11T00:00:00.000,Pcr,11,724,Otro,Nan
9,10,2020-03-12T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,36,F,Importado,Leve,...,2020-03-06T00:00:00.000,Nan,2020-03-12T00:00:00.000,2020-03-21T00:00:00.000,2020-03-12T00:00:00.000,Pcr,11,724,Otro,Nan
10,11,2020-03-11T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,42,F,Importado,Leve,...,2020-03-06T00:00:00.000,Nan,2020-03-12T00:00:00.000,2020-03-31T00:00:00.000,2020-03-12T00:00:00.000,Pcr,11,724,Otro,Nan


In [46]:
# valores NaN
if dfcovid19.isna().sum().sum() > 0:
    dfcovid19.fillna(value='-', inplace=True)

dfcovid19.head()

,ID DE CASO,FECHA DE NOTIFICACION,CODIGO DIVIPOLA,CIUDAD DE UBICACION,DEPARTAMENTO O DISTRITO,ATENCION,EDAD,SEXO,TIPO,ESTADO,...,FIS,FECHA DE MUERTE,FECHA DIAGNOSTICO,FECHA RECUPERADO,FECHA REPORTE WEB,TIPO RECUPERACION,CODIGO DEPARTAMENTO,CODIGO PAIS,PERTENENCIA ETNICA,NOMBRE GRUPO ETNICO
0,1,2020-03-02T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Recuperado,19,F,Importado,Leve,...,2020-02-27T00:00:00.000,Nan,2020-03-06T00:00:00.000,2020-03-13T00:00:00.000,2020-03-06T00:00:00.000,Pcr,11,380,Otro,Nan
1,2,2020-03-06T00:00:00.000,76111,Guadalajara De Buga,Valle Del Cauca,Recuperado,34,M,Importado,Leve,...,2020-03-04T00:00:00.000,Nan,2020-03-09T00:00:00.000,2020-03-19T00:00:00.000,2020-03-09T00:00:00.000,Pcr,76,724,Otro,Nan
2,3,2020-03-07T00:00:00.000,5001,Medellín,Antioquia,Recuperado,50,F,Importado,Leve,...,2020-02-29T00:00:00.000,Nan,2020-03-09T00:00:00.000,2020-03-15T00:00:00.000,2020-03-09T00:00:00.000,Pcr,5,724,Otro,Nan
3,4,2020-03-09T00:00:00.000,5001,Medellín,Antioquia,Recuperado,55,M,Relacionado,Leve,...,2020-03-06T00:00:00.000,Nan,2020-03-11T00:00:00.000,2020-03-26T00:00:00.000,2020-03-11T00:00:00.000,Pcr,5,-,Otro,Nan
4,5,2020-03-09T00:00:00.000,5001,Medellín,Antioquia,Recuperado,25,M,Relacionado,Leve,...,2020-03-08T00:00:00.000,Nan,2020-03-11T00:00:00.000,2020-03-23T00:00:00.000,2020-03-11T00:00:00.000,Pcr,5,-,Otro,Nan


In [38]:
# df de ciudades
# limpiar NaN
# convertir fechas .type
# explorar columnas - tipo de dato, datos NaN, calidad del dato

NameError: name 'df' is not defined